In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [48]:
data = pd.read_csv('/Users/elvisyan/Desktop/train.csv')

In [49]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [50]:
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)



/var/folders/9_/nngl5mtd5fj36m6d20z7p0pr0000gn/T/ipykernel_61676/3357712235.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
/var/folders/9_/nngl5mtd5fj36m6d20z7p0pr0000gn/T/ipykernel_61676/3357712235.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [51]:
# Encode categorical features
label_enc = LabelEncoder()
data['Sex'] = label_enc.fit_transform(data['Sex'])
data['Embarked'] = label_enc.fit_transform(data['Embarked'])


In [52]:

# Features and target
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = data['Survived']


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [54]:
def custom_ensemble(X_train, y_train, X_test, n_estimators=10):
    np.random.seed(42)
    models = []
    predictions = []
        # Generate random samples and train a model for each
    for _ in range(n_estimators):
        # Bootstrap sampling
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_sample, y_sample = X_train.iloc[indices], y_train.iloc[indices]
        # Train a model
        model = DecisionTreeClassifier()
        model.fit(X_sample, y_sample)
        models.append(model)
         # Store predictions for aggregation
        pred = model.predict(X_test)
        predictions.append(pred)
    # Aggregate predictions (majority voting)
    predictions = np.array(predictions)
    aggregated_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

    return aggregated_predictions, models

In [55]:
# Use the custom ensemble model
y_pred_custom, custom_models = custom_ensemble(X_train, y_train, X_test)
custom_accuracy = accuracy_score(y_test, y_pred_custom)
print(f"Custom Ensemble Model Accuracy: {custom_accuracy:.4f}")

Custom Ensemble Model Accuracy: 0.7598


In [58]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [63]:

# Bagging Classifier
bagging_clf = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_clf.fit(X_train, y_train)
y_pred_bagging = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)
print(f"Bagging Classifier Accuracy: {bagging_accuracy:.4f}")


Bagging Classifier Accuracy: 0.7709


In [ ]:

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=10, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Classifier Accuracy: {rf_accuracy:.4f}")

Random Forest Classifier Accuracy: 0.7989


In [64]:
# Single Decision Tree
single_tree = DecisionTreeClassifier(random_state=42)
single_tree.fit(X_train, y_train)
y_pred_single = single_tree.predict(X_test)
single_accuracy = accuracy_score(y_test, y_pred_single)
print(f"Single Decision Tree Accuracy: {single_accuracy:.4f}")

Single Decision Tree Accuracy: 0.7821


In [65]:
# Measure time for the custom ensemble
%timeit custom_ensemble(X_train, y_train, X_test)

# Measure time for Bagging Classifier
%timeit bagging_clf.fit(X_train, y_train)

# Measure time for Random Forest Classifier
%timeit rf_clf.fit(X_train, y_train)

# Measure time for Single Decision Tree
%timeit single_tree.fit(X_train, y_train)

59.8 ms ± 7.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.5 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.6 ms ± 708 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.38 ms ± 530 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
from scipy.stats import mode
import time

In [67]:
data = sns.load_dataset('titanic')

In [68]:
df = data.drop(['deck','class','who','adult_male','embark_town','alive','alone'],axis=1)

In [69]:
df['age'] = df['age'].fillna(df.groupby('sex')['age'].transform('mean'))

In [70]:
df['fam'] = df['parch'] + df['sibsp']

In [71]:
df = pd.get_dummies(df, columns=['sex','embarked','pclass'], dtype=int)

In [72]:
y = df['survived']
X = df.drop('survived', axis=1)

In [75]:
all_columns = df.columns.to_list()
random_columns = [np.random.choice(all_columns, 3, replace=True).tolist() for i in range (10)]

In [76]:
random_columns


[['parch', 'fare', 'fam'],
 ['survived', 'parch', 'embarked_S'],
 ['pclass_1', 'embarked_Q', 'pclass_2'],
 ['fare', 'sex_female', 'pclass_2'],
 ['fare', 'pclass_2', 'pclass_2'],
 ['survived', 'embarked_S', 'sex_female'],
 ['sex_male', 'pclass_2', 'sex_female'],
 ['embarked_S', 'pclass_3', 'pclass_2'],
 ['parch', 'sex_male', 'fare'],
 ['embarked_Q', 'embarked_C', 'pclass_1']]

In [77]:
def build_decision_trees(X_train, y_train, num_models=10, num_columns=3):
    all_columns = X_train.columns.to_list()
    models = []
    for i in range(num_models):
        selected_columns = np.random.choice(all_columns, num_columns, replace=True).tolist()
        X_train_subset = X_train[selected_columns]
        tree = DecisionTreeClassifier(random_state=42)
        tree.fit(X_train_subset, y_train)
        models.append((tree, selected_columns))
    return models

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
models = build_decision_trees(X_train, y_train, num_models=10, num_columns=3)